In [1]:
# [1]: Importar librerías necesarias
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, split, col
from pyspark.sql.types import StringType

# [2]: Crear sesión de Spark
spark = SparkSession.builder \
    .appName("KafkaWordCount") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.0") \
    .getOrCreate()

# [3]: Leer stream desde Kafka
kafka_df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "kafka:9092") \
    .option("subscribe", "actividad-topic") \
    .load()

In [2]:
# Convertir el valor de Kafka a STRING
kafka_df_string = kafka_df.selectExpr("CAST(value as STRING)")



# Escribir el stream a memoria
query = kafka_df_string.writeStream \
    .outputMode("append") \
    .format("memory") \
    .queryName("messages_table") \
    .start()

In [3]:
import time

while True:
    try:
        from IPython.display import clear_output
        clear_output(wait=True)
        
        print("Consultando resultados en vivo...")
        results_df = spark.sql("SELECT * FROM messages_table")
        results_df.show()
        
        time.sleep(5)
        
    except KeyboardInterrupt:
        print("Consulta en vivo detenida.")
        break

Consultando resultados en vivo...
+--------------------+
|               value|
+--------------------+
|El topico fue creado|
|Este es mi primer...|
|El producer manda...|
|     Hola Soy Marcos|
|             Saludos|
|Este es mi primer...|
|Este es mi primer...|
|             Saludos|
|             Saludos|
|     Hola Soy Marcos|
| El consumer los lee|
|El producer manda...|
+--------------------+

Consulta en vivo detenida.
